In [47]:
import pandas as pd
import re
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import torch

In [ ]:
df=pd.read_csv('master_dataset_updated.csv')

In [ ]:
df.columns

Index(['id', 'title', 'description', 'publishedAt', 'tags', 'categoryId',
       'defaultLanguage', 'defaultAudioLanguage', 'thumbnail_default',
       'thumbnail_high', 'duration', 'viewCount', 'likeCount', 'commentCount',
       'privacyStatus', 'channel_id', 'channel_title', 'channel_description',
       'channel_country', 'channel_thumbnail', 'channel_subscriberCount',
       'channel_videoCount', 'is_transcript_available', 'transcript'],
      dtype='object')

In [ ]:
def clean_text(text):
    if text is None or pd.isna(text):
        return ""
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = ' '.join(text.split())
    return text

In [ ]:
clean_text("Why studying bees can teach us about human loneliness | Sarah Kocher | TEDxNewEngland")

'why studying bees can teach us about human loneliness sarah kocher tedxnewengland'

In [ ]:
df['title_cleaned'] = df['title'].apply(clean_text)
df['description_cleaned'] = df['description'].apply(clean_text)
df['transcript_cleaned'] = df['transcript'].apply(clean_text)

In [ ]:
df["title_cleaned"].head()

0    scientists cant explain whats happening on thi...
1     million pirate treasure found on the ocean floor
2       if you see a rubber band on your door act fast
3    cleopatras tomb found egypts greatest discover...
4    scientists just solved the mystery of killer w...
Name: title_cleaned, dtype: object

In [ ]:
df['duration_seconds'] = pd.to_timedelta(df['duration'], errors='coerce').dt.total_seconds()

In [ ]:
df[['duration','duration_seconds']].head()

,duration,duration_seconds
0,PT3H59M26S,14366.0
1,PT8M50S,530.0
2,PT4H3M28S,14608.0
3,PT3H4M25S,11065.0
4,PT9M14S,554.0


In [ ]:
df.to_csv('master_dataset_processed.csv', index=False)

In [ ]:
df.columns

Index(['id', 'title', 'description', 'publishedAt', 'tags', 'categoryId',
       'defaultLanguage', 'defaultAudioLanguage', 'thumbnail_default',
       'thumbnail_high', 'duration', 'viewCount', 'likeCount', 'commentCount',
       'privacyStatus', 'channel_id', 'channel_title', 'channel_description',
       'channel_country', 'channel_thumbnail', 'channel_subscriberCount',
       'channel_videoCount', 'is_transcript_available', 'transcript',
       'title_cleaned', 'description_cleaned', 'transcript_cleaned',
       'duration_seconds'],
      dtype='object')

In [2]:
from sentence_transformers import SentenceTransformer

In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\Folders\INFOSYS\sentence_fix_env\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Folders\INFOSYS\sentence_fix_env\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\Folders\INFOSYS\sentence_fix_env\Lib\site-packages\ipykernel\kernelapp.py", line 758, in start
    self.io_loop.start()
  File "d:\Fo

In [3]:
df_e=pd.read_csv('master_dataset_processed.csv')

In [6]:
df_e.columns

Index(['id', 'title', 'description', 'publishedAt', 'tags', 'categoryId',
       'defaultLanguage', 'defaultAudioLanguage', 'thumbnail_default',
       'thumbnail_high', 'duration', 'viewCount', 'likeCount', 'commentCount',
       'privacyStatus', 'channel_id', 'channel_title', 'channel_description',
       'channel_country', 'channel_thumbnail', 'channel_subscriberCount',
       'channel_videoCount', 'is_transcript_available', 'transcript',
       'title_cleaned', 'description_cleaned', 'transcript_cleaned',
       'duration_seconds'],
      dtype='object')

In [28]:
np.set_printoptions(legacy='1.21')
print(f"NumPy version: {np.__version__}")
import numpy as np

NumPy version: 2.3.3


In [29]:
df_e['combined_title_transcript'] = df_e['title_cleaned']+" "+df_e['transcript_cleaned']

In [43]:
def chunk_text(text, max_tokens=500, overlap=50):
    if pd.isna(text) or text == "":
        return []
    words = text.split()
    chunks = []
    
    start = 0
    while start < len(words):
        end = start + max_tokens
        chunk = ' '.join(words[start:end])
        chunks.append(chunk)
        start = end - overlap
        if start >= len(words):
            break
    
    return chunks

In [49]:
def get_embeddings_pytorch(text):
    """Use pure PyTorch to avoid numpy issues"""
    if pd.isna(text) or text == "":
        return torch.zeros(384, dtype=torch.float32)
    
    chunks = chunk_text(text)
    if not chunks:
        return torch.zeros(384, dtype=torch.float32)
    
    try:
        # Get embeddings as tensor
        chunk_embeddings = model.encode(chunks, convert_to_tensor=True)
        
        # Calculate mean using PyTorch
        if len(chunk_embeddings.shape) > 1:
            final_embedding = chunk_embeddings.mean(dim=0)
        else:
            final_embedding = chunk_embeddings
            
        return final_embedding.cpu()  # Ensure it's on CPU
        
    except Exception as e:
        print(f"Error processing text: {e}")
        return torch.zeros(384, dtype=torch.float32)

In [50]:
df_e['e_title_trans_tensor'] = df_e['combined_title_transcript'].apply(get_embeddings_pytorch)
df_e['e_description_tensor'] = df_e['description_cleaned'].apply(get_embeddings_pytorch)

In [51]:
df_e.columns

Index(['id', 'title', 'description', 'publishedAt', 'tags', 'categoryId',
       'defaultLanguage', 'defaultAudioLanguage', 'thumbnail_default',
       'thumbnail_high', 'duration', 'viewCount', 'likeCount', 'commentCount',
       'privacyStatus', 'channel_id', 'channel_title', 'channel_description',
       'channel_country', 'channel_thumbnail', 'channel_subscriberCount',
       'channel_videoCount', 'is_transcript_available', 'transcript',
       'title_cleaned', 'description_cleaned', 'transcript_cleaned',
       'duration_seconds', 'combined_title_transcript', 'e_title_trans_tensor',
       'e_description_tensor'],
      dtype='object')

In [52]:
df_e.to_csv('master_dataset_embedded.csv', index=False)